In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Dataset, Datastore, ComputeTarget, Experiment, ScriptRunConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep='\n')

In [ ]:
# create an ML experiment
exp = None

# create a directory
script_folder = './keras-mnist-fashion'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6s_v3', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


In [ ]:
data_urls = ['https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion']
fashion_ds = None

# list the files referenced by fashion_ds
fashion_ds.to_path()


In [ ]:
from azureml.data import OutputFileDatasetConfig

datastore=workspace.get_default_datastore()
prepared_fashion_ds = None


In [ ]:
prep_step = None

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-core
  - azureml-dataset-runtime
  - keras==2.4.3
  - tensorflow==2.4.3
  - numpy
  - scikit-learn
  - pandas
  - matplotlib

In [ ]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-env', file_path = './conda_dependencies.yml')

In [ ]:
train_src = ScriptRunConfig(source_directory=script_folder,
                            script='train.py',
                            compute_target=compute_target,
                            environment=keras_env)

In [ ]:


train_step = None


In [ ]:


# build pipeline & run experiment
pipeline = None
run = exp.submit(pipeline)



In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.find_step_run('train step')[0].get_metrics()

In [ ]:
# get input datasets
prep_step = run.find_step_run('prepare step')[0]
inputs = prep_step.get_details()['inputDatasets']
input_dataset = inputs[0]['dataset']

# list the files referenced by input_dataset
input_dataset.to_path()

In [ ]:
fashion_ds = input_dataset.register(workspace = workspace,
                                    name = 'fashion_ds',
                                    description = 'image and label files from fashion mnist',
                                    create_new_version = True)
fashion_ds


In [ ]:


run.find_step_run('train step')[0].register_model(None)

